In [7]:
import django_initializer
from backend.pymongo.mongodb import db
from datetime import datetime
import arrow
from database.lss.campaign import Campaign
from backend.pymongo.mongodb import db
from channels.layers import get_channel_layer
from asgiref.sync import async_to_sync
from time import sleep

In [10]:
def do():
    a = arrow.now()
    after_15min = arrow.utcnow().replace(second=0, microsecond=0).shift(minutes=+15)
    after_16min = after_15min.shift(minutes=+1)
    after_1hour = arrow.utcnow().replace(second=0, microsecond=0).shift(minutes=+60)
    after_61mins = after_1hour.shift(minutes=+1)
    campaings_start_after_15mins = list(db.api_campaign.aggregate([
        {
            "$match":{
                "start_at":{
                    "$gte": after_15min.datetime,
                    "$lt": after_16min.datetime
                }
            }
        },
        {
            "$project":{
                "_id":0,
                "user_subscription_id":1,
                "title":1,
                "remind_time": "15 mins"
            }
        }
    ]))
    campaings_start_after_1hour = list(db.api_campaign.aggregate([
        {
            "$match":{
                "start_at":{
                    "$gte": after_1hour.datetime,
                    "$lt": after_61mins.datetime
                }
            }
        },
        {
            "$project":{
                "_id":0,
                "user_subscription_id":1,
                "title":1,
                "remind_time": "an hour"
            }
        }
    ]))
    campaigns = campaings_start_after_15mins + campaings_start_after_1hour
    print(campaigns)
    channel_layer = get_channel_layer()
    b = arrow.now()
    # print(b-a)
    for campaign in campaigns:
        user_subscription_id = campaign['user_subscription_id']
        title = campaign['title']
        remind_time = campaign['remind_time']
        async_to_sync(channel_layer.group_send)(f"user_subscription_{user_subscription_id}", {"type": "notification_message","data":{"message":{"title": title, "remind_time": remind_time}}})

In [11]:

while True:
    do()
    sleep(60)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


KeyboardInterrupt: 